In [330]:
import pandas as pd
import numpy as np


## 1. Импорт и  первичный анализ данных

In [331]:
sample_a = pd.read_csv('data/sample_a.zip')
sample_b = pd.read_csv('data/sample_b.zip')
sample_c = pd.read_csv('data/sample_c.zip')

display(sample_a.head(), sample_b.head(), sample_c.head())

,user_id,item_id,action_id
0,84636,360,1
1,21217,9635,1
2,13445,8590,1
3,38450,5585,1
4,14160,2383,0


,user_id,item_id,action_id
0,118375,4105,1
1,107569,8204,1
2,175990,880,1
3,160582,9568,0
4,123400,4000,1


,user_id,item_id,action_id
0,274623,2863,1
1,265472,343,1
2,242779,6009,0
3,275009,2184,1
4,268104,3134,2


In [332]:
display(sample_a.shape, sample_b.shape, sample_c.shape)

(1188912, 3)

(1198438, 3)

(1205510, 3)

In [333]:
display(sample_a.info(), sample_b.info(), sample_c.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188912 entries, 0 to 1188911
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1188912 non-null  int64
 1   item_id    1188912 non-null  int64
 2   action_id  1188912 non-null  int64
dtypes: int64(3)
memory usage: 27.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198438 entries, 0 to 1198437
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1198438 non-null  int64
 1   item_id    1198438 non-null  int64
 2   action_id  1198438 non-null  int64
dtypes: int64(3)
memory usage: 27.4 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205510 entries, 0 to 1205509
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1205510 non-null  int64
 1   item_id    1205510 non-null  int64
 2   action_id  1205510 no

None

None

None

In [334]:
display(sample_a.isna().sum(), sample_b.isna().sum(), sample_c.isna().sum())

user_id      0
item_id      0
action_id    0
dtype: int64

user_id      0
item_id      0
action_id    0
dtype: int64

user_id      0
item_id      0
action_id    0
dtype: int64

In [335]:
item_prices = pd.read_csv('data/item_prices.zip')
item_prices.head()

,item_id,item_price
0,338,1501
1,74,647
2,7696,825
3,866,875
4,5876,804


In [336]:
item_prices.shape

(1000, 2)

In [337]:
item_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   item_id     1000 non-null   int64
 1   item_price  1000 non-null   int64
dtypes: int64(2)
memory usage: 15.8 KB


In [338]:
item_prices.isna().sum()

item_id       0
item_price    0
dtype: int64

## 2. Проверка, что нет ситуаций, когда происходит покупка/клик без действия просмотра. Удаление дублей.

Первым делом удалим дубликаты при их наличии. 

In [339]:
sample_a.drop_duplicates(inplace=True)
sample_a.shape

(1188912, 3)

In [340]:
sample_b.drop_duplicates(inplace=True)
sample_b.shape

(1198438, 3)

In [341]:
sample_c.drop_duplicates(inplace=True)
sample_c.shape

(1205510, 3)

Для датафрейма с ценами есть нюанс. Нам нет возможности использовать несолько цен для одного товара

In [342]:
item_prices['duplicated'] = item_prices.duplicated(subset='item_id')
# item_prices[item_prices.duplicated == True]
duplicates = item_prices[item_prices['duplicated'] == True].sort_values('item_id')
duplicates.head()

,item_id,item_price,duplicated
977,338,1047,True
797,338,1151,True
910,533,294,True
851,1173,978,True
107,1462,870,True


In [343]:
for elem in set(duplicates.item_id):
    price_mean = item_prices.loc[item_prices['item_id'] == elem, 'item_price'].mean()
    item_prices.loc[item_prices['item_id'] == elem, 'item_price'] = price_mean

item_prices[item_prices.item_id == 338].item_price = price_mean
item_prices[item_prices.item_id == 338]


C:\Users\User\AppData\Local\Temp\ipykernel_25340\1715058452.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1135.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  item_prices.loc[item_prices['item_id'] == elem, 'item_price'] = price_mean
C:\Users\User\AppData\Local\Temp\ipykernel_25340\1715058452.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_prices[item_prices.item_id == 338].item_price = price_mean


,item_id,item_price,duplicated
0,338,1233.0,False
797,338,1233.0,True
977,338,1233.0,True


In [344]:
item_prices.drop_duplicates(subset='item_id', inplace=True)
item_prices.drop('duplicated', axis=1, inplace=True)
item_prices.shape

(955, 2)

Проверим наборы данных на наличие записей о покупке товара без просмотре. При сведении в сводную таблицу по 'user_id' и 'item_id' у товара, который купили без просмотра сумма по столбцу'action_id' должна быть равна 2.

In [345]:
df = sample_a.pivot_table(index=['user_id', 'item_id'], aggfunc='sum')
df.action_id.unique()

array([1, 3], dtype=int64)

In [346]:
df = sample_b.pivot_table(index=['user_id', 'item_id'], aggfunc='sum')
df.action_id.unique()

array([1, 3], dtype=int64)

In [347]:
df = sample_c.pivot_table(index=['user_id', 'item_id'], aggfunc='sum')
df.action_id.unique()

array([1, 3], dtype=int64)

В наборах данных отсутствуют данные о товарах купленных без просмотра.

## 3. Рассчитать метрики по датасетам, провести общее сравнение метрик.

Предлагаю первым делом привести все выборки к единому количеству наблюдений. Наименьшее количество наблюдений в выборке sample_a - 1188912.

In [348]:
sample_b = sample_b[:1188912]
sample_c = sample_c[:1188912]

sample_b.shape, sample_c.shape

((1188912, 3), (1188912, 3))

Расчтиываем следующие метрики по датасетам:

*ctr (отношение кликов к просмотрам товаров)*

In [349]:
ctr_a = sample_a[sample_a.action_id == 0].action_id.count() / sample_a[sample_a.action_id == 1].action_id.count()
ctr_b = sample_b[sample_b.action_id == 0].action_id.count() / sample_b[sample_b.action_id == 1].action_id.count()
ctr_c = sample_c[sample_c.action_id == 0].action_id.count() / sample_c[sample_c.action_id == 1].action_id.count()

print('Метрика ctr для выборки sample_a: ', round(ctr_a, 2))
print('Метрика ctr для выборки sample_b: ', round(ctr_b, 2))
print('Метрика ctr для выборки sample_c: ', round(ctr_c, 2))

Метрика ctr для выборки sample_a:  0.2
Метрика ctr для выборки sample_b:  0.16
Метрика ctr для выборки sample_c:  0.21


*purchase rate (отношение покупок к просмотрам товаров)*

In [350]:
purchase_rate_a = sample_a[sample_a.action_id == 2].action_id.count() / sample_a[sample_a.action_id == 1].action_id.count()
purchase_rate_b = sample_b[sample_b.action_id == 2].action_id.count() / sample_b[sample_b.action_id == 1].action_id.count()
purchase_rate_c = sample_c[sample_c.action_id == 2].action_id.count() / sample_c[sample_c.action_id == 1].action_id.count()

print('Метрика purchase rate для выборки sample_a: ', round(purchase_rate_a, 2))
print('Метрика purchase rate для выборки sample_b: ', round(purchase_rate_b, 2))
print('Метрика purchase rate для выборки sample_c: ', round(purchase_rate_c, 2))

Метрика purchase rate для выборки sample_a:  0.05
Метрика purchase rate для выборки sample_b:  0.1
Метрика purchase rate для выборки sample_c:  0.06


*gmv (оборот, сумма произведений количества покупок на стоимость покупки), где считаем 1 сессию за 1 точку (1 сессия на 1 пользователя)*

In [351]:
sample_a_price = sample_a.merge( item_prices, how='left', on='item_id', validate='many_to_one')
sample_a_price

,user_id,item_id,action_id,item_price
0,84636,360,1,1896.0
1,21217,9635,1,1699.0
2,13445,8590,1,846.0
3,38450,5585,1,1556.0
4,14160,2383,0,1956.0
...,...,...,...,...
1188907,22999,2401,1,1670.0
1188908,23700,4654,0,1080.0
1188909,18842,3707,1,656.0
1188910,32732,9198,1,1307.0


In [352]:
from statsmodels.stats.proportion import proportions_ztest
alpha = 0.05 # уровень значимости
# вычисляем значение p-value для Z-теста для пропорций
_, p_value = proportions_ztest(
count=converted_piv['sum'], # число "успехов"
nobs=converted_piv['count'], # общее число наблюдений
alternative='two-sided',
)
# выводим результат на экран
print('p-value: ', round(p_value, 3))
# сравниваем полученное p-value с уровнем значимости
if (p_value < alpha):
print("Отвергаем нулевую гипотезу в пользу альтернативной")
else:
print("У нас нет оснований отвергнуть нулевую гипотезу")

IndentationError: expected an indented block after 'if' statement on line 12 (1939934693.py, line 13)